# 模型选择（过拟合欠拟合的出现和解决）
我们在训练深度学习模型时，期望使用训练数据训练的这个模型能在测试数据上也能得到很好的表现，这个就是期望泛化误差减小。（_泛化误差:_ 模型在任意测试数据上的输出结果和真实结果的差距的期望，通常使用测试集上的误差来代表）

为了得到这样一个模型，其中的主要影响因素是模型复杂度，输入数据量，训练时间，训练方法等。但是由于我们没有确切的方式定义这些因素是如何影响的，因此，我们需要各种各样的方法来选择模型。

## 模型选择的方法
选择模型时，我们主要参考两个指标。一个是训练误差，一个是验证误差，这里不能使用泛化误差是因为这样会导致泛化误差有偏，不能反映真实的泛化误差。
训练误差和验证误差分别是模型在训练数据集和验证数据集上的误差，这个误差可以用交叉熵损失函数，MSE损失函数等多种损失函数计算得出
* 验证数据集进行验证：
验证集通过在整体数据中抽出一定比例数据组成，其不参与模型的训练过程，在模型使用训练数据训练完成后，我们通过其在验证集上的精度进行模型的选择。
* k折验证法：
有时，我们的数据不是非常充足，因此为了减少验证集对数据的浪费，我们采用K折验证法：
   1. 我们将整个数据分成k份，其中（K-1）份代表训练集，1份代表验证集，同时我们要遍历所有的组合方式，也就生成K份不同的数据集
   2. 模型分别基于K份数据集中的训练集进行训练，对应的验证集进行验证，获得的验证精度进行平均即获得这个模型的精度。

## 训练模型的过拟合和欠拟合
训练出来的模型一般存在两类问题，导致最终的验证结果不是最理想，因此我们需要尽量避免。
<div align=center>

![Image Name](Pics/OVERFIT.png)
</div>

_欠拟合_：训练误差无法达到较小，说明此时在训练数据上模型都不能正确拟合。
_过拟合_：当训练误差已经较小，但是验证误差和训练误差相差较大，说明模型仅仅拟合到训练数据，该模型并不能泛化。

影响欠拟合和过拟合的因素：
影响模型有没有拟合/过拟合的因素是多方面的，包括模型模型复杂度，数据量，训练时间，训练方法等。
* 模型复杂度：模型复杂度可以受到参数量，网络的广度深度的影响，其最终等效为这个模型能描述的所有的函数的集合。一个过大的模型在一定时间的训练下很容易过度拟合
* 数据量： 数据量不仅仅是数据的多少，其还应该具有足够代表性和多样化。
* 训练时间&训练方法：合适的训练方法和恰当的训练时间可以避免模型过拟合/欠拟合


- 过拟合和欠拟合的解决方法
为了解决过拟合/欠拟合，我们需要对模型的复杂度进行限制/增加更多的数据，下面提出两种解决过拟合的方法
* 权重衰减和正则化：
    在使用SGD优化方法时，正则化和权重衰减具有一致性。其中L2正则化是在模型的损失上添加一个权重的二范数作为惩罚项，从而限制参数的大小。
    具体表达式以线性回归模型的损失函数为例：
    <div align=center>

    ![Untitled/Untitled.png](Pics/linearloss.png)

    </div>

    正则项则是参数的二范数:
    <div align=center>

    ![Untitled/Untitled.png](Pics/loss_reg.png)

    </div>

    最终，根据SGD的参数更新方式，我们可以得到如下更新公式：
    <div align=center>

    ![Untitled/Untitled.png](Pics/SGDupdate.png)

    </div>

    可以发现，添加了L2正则项的损失函数促使更新公式在权重项之前乘以了一个小于1的系数，也就是等效于权重衰减设置为lr*lambda

* inverted dropout 丢弃法：

   dropout的思想是在前向传播的时候，设置一概率p，使得每个神经元有p的概率输出为0，很明显，这样的方式可以达到网络稀疏化的效果，从而削弱了网络的表达能力，即复杂度。从而是一种有效的防止过拟合的方式，下面我们看看其具体是怎么运行的：

    1. 在前向传播的时候，设置一个存在权重n，其有p的概率为0，1-p的概率为1，从而一个神经元输出为：

       这里我们发现其输出的期望还是等于自身：

       也就是增加了dropout的网络的输出平均值与没有增加dropout一致，而我们又知道dropout每次代表不同的网络结构，因此对增加了dropout的网络输出求平均即为对不同网络结构求平均，这样我们实现了一个网络结构和一次训练，但是求出的平均值等效于同时训练多个网络得到的平均值。这一种方法也有效地减少了过拟合。

    2. 在训练过程中我们不断使用不同的数据重复上述步骤

    3. 我们在测试是去除缩放因子，并对网络进行测试，由于第一步我们知道添加dropout与否的网络的平均值相同，那么我们在测试时去除缩放因子就保证了和训练的平均值相同。即我们测试了我们训练的网络。

   那么dropout在网络结构层次上又是怎么影响的呢？
   其作者认为，在训练神经网络时，过拟合现象起源于神经元之间的co-adpatation 相互适应，也就是某个神经元在给定某些固定特征时，其强行根据一些不正确的特征组合出最终的正确答案，因此dropout就打破了特征和输出的固定联系，从而促使网络根据一个更加general的场景做决定。


